# Quick Maths with Matrices!
---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Quick-Maths-with-Matrices!" data-toc-modified-id="Quick-Maths-with-Matrices!-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Quick Maths with Matrices!</a></span><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Test-Framework" data-toc-modified-id="Test-Framework-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Test Framework</a></span></li></ul></li><li><span><a href="#Optimizing-Matrix-Multiplications" data-toc-modified-id="Optimizing-Matrix-Multiplications-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Optimizing Matrix Multiplications</a></span><ul class="toc-item"><li><span><a href="#For-Loop" data-toc-modified-id="For-Loop-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>For Loop</a></span></li><li><span><a href="#Array-Slicing" data-toc-modified-id="Array-Slicing-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Array Slicing</a></span></li><li><span><a href="#Improvement-with-array-slicing" data-toc-modified-id="Improvement-with-array-slicing-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Improvement with array slicing</a></span></li><li><span><a href="#Array-Broadcasting" data-toc-modified-id="Array-Broadcasting-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Array Broadcasting</a></span></li><li><span><a href="#Improvement-with-array-broadcasting" data-toc-modified-id="Improvement-with-array-broadcasting-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Improvement with array broadcasting</a></span></li><li><span><a href="#Einstein-Sum" data-toc-modified-id="Einstein-Sum-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Einstein Sum</a></span></li><li><span><a href="#Improvement-with-einstein-sum" data-toc-modified-id="Improvement-with-einstein-sum-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Improvement with einstein sum</a></span></li><li><span><a href="#Linear-Algebra-Libraries" data-toc-modified-id="Linear-Algebra-Libraries-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Linear Algebra Libraries</a></span></li><li><span><a href="#Improvement-with-linear-algebra-libraries" data-toc-modified-id="Improvement-with-linear-algebra-libraries-2.9"><span class="toc-item-num">2.9&nbsp;&nbsp;</span>Improvement with linear algebra libraries</a></span></li></ul></li></ul></div>

## Import Libraries

In [1]:
import torch
import timeit
import operator
from functools import partial

## Test Framework

In [2]:
def test(a, b, compare, compare_name=None):
    if compare_name is None:
        compare_name = compare.__name__
    assert compare(a, b),\
    f"{compare_name} check failed:\n{a}\n{b}"

def test_equality(a, b):
    test(a, b, operator.eq, "Equality")

def test_approximately(a, b):
    allclose = partial(torch.allclose, atol=1e-5, rtol=1e-03)
    if not isinstance(a, torch.Tensor) or not isinstance(b, torch.Tensor):
        a = torch.tensor(a)
        b = torch.tensor(b)
    test(a, b, allclose, "Approximate Equality")

In [3]:
test_equality(1e-5,1e-5)

In [4]:
test_approximately(1e-5, 1e-6)

# Optimizing Matrix Multiplications

**Toy Variables**

In [5]:
a = torch.tensor([[1.,2.,1.],
                  [2.,3.,2.],
                  [3.,1.,3.]])

b = torch.tensor([[1., 2.],
                  [2., 1.],
                  [1., 2.]])

In [6]:
a@b

tensor([[ 6.,  6.],
        [10., 11.],
        [ 8., 13.]])

**Test Variables**

In [7]:
A = torch.randn([64,32])
B = torch.randn([32,64])

In [8]:
(A@B).shape

torch.Size([64, 64])

## For Loop

In [9]:
def matmul(A,B):
    A_rows, A_cols = A.shape
    B_rows, B_cols = B.shape
    assert A_cols==B_rows,\
    f"Inner dimensions must match: {A_cols} not equal to {B_rows}"
    C = torch.zeros([A_rows, B_cols])
    for i in range(A_rows):
        for j in range(B_cols):
            for k in range(A_cols):
                C[i,j] += A[i,k] * B[k,j]
    return C

In [10]:
matmul(a,b)

tensor([[ 6.,  6.],
        [10., 11.],
        [ 8., 13.]])

In [11]:
test_approximately(matmul(A, B), (A@B))

In [12]:
matmul_loop_time = timeit.timeit(partial(matmul,A,B), number=10)
matmul_loop_time

15.216818227999966

## Array Slicing

In [13]:
def matmul(A,B):
    A_rows, A_cols = A.shape
    B_rows, B_cols = B.shape
    assert A_cols==B_rows,\
    f"Inner dimensions must match: {A_cols} not equal to {B_rows}"
    C = torch.zeros([A_rows, B_cols])
    for i in range(A_rows):
        for j in range(B_cols):
            C[i,j] += (A[i,:]*B[:,j]).sum()
    return C

In [14]:
matmul(a,b)

tensor([[ 6.,  6.],
        [10., 11.],
        [ 8., 13.]])

In [15]:
test_approximately(matmul(A,B), (A@B))

In [16]:
matmul_slice_time = timeit.timeit(partial(matmul,A,B), number=10)

## Improvement with array slicing
How fast is matrix multiplication with array slicing in two nested loops compared to element wise product in three nested loops?

In [17]:
matmul_loop_time, matmul_slice_time

(15.216818227999966, 0.642857911999954)

In [18]:
print(f"Basic matrix multiplication of a {A.shape[0]}x{A.shape[1]} matrix "
      f"with {B.shape[0]}x{B.shape[1]} matrix in python "
      f"takes about {matmul_loop_time:.0f} seconds!")

Basic matrix multiplication of a 64x32 matrix with 32x64 matrix in python takes about 15 seconds!


In [19]:
loop_vs_slice = matmul_loop_time/matmul_slice_time

In [20]:
print(f"Array slicing is about {loop_vs_slice:.0f} times faster than element wise product in three nested loops")

Array slicing is about 24 times faster than element wise product in three nested loops


## Array Broadcasting

In [21]:
def matmul(A,B):
    A_rows, A_cols = A.shape
    B_rows, B_cols = B.shape
    assert A_cols==B_rows,\
    f"Inner dimensions must match: {A_cols} not equal to {B_rows}"
    C = torch.zeros([A_rows, B_cols])
    for i in range(A_rows):
        C[i,:] += (A[i,:]*B.t()).sum(1)
    return C

In [22]:
matmul(a,b)

tensor([[ 6.,  6.],
        [10., 11.],
        [ 8., 13.]])

In [23]:
test_approximately(matmul(A, B), (A@B))

In [24]:
matmul_broadcast_time = timeit.timeit(partial(matmul,A,B), number=10)

## Improvement with array broadcasting
How fast is matrix multiplication with broadcasting compared to array slicing in two nested loops?

In [25]:
slice_vs_broadcast = matmul_slice_time/matmul_broadcast_time
slice_vs_broadcast

50.74959926174802

In [26]:
print(f"Array broadcasting is about {slice_vs_broadcast:.0f} times faster than array slicing in two nested loops")

Array broadcasting is about 51 times faster than array slicing in two nested loops


## Einstein Sum

In [27]:
def matmul(A,B):
    A_rows, A_cols = A.shape
    B_rows, B_cols = B.shape
    assert A_cols==B_rows,\
    f"Inner dimensions must match: {A_cols} not equal to {B_rows}"
    C = torch.zeros([A_rows, B_cols])
    C = torch.einsum("ik,kj->ij", A, B)
    return C

In [28]:
matmul(a,b)

tensor([[ 6.,  6.],
        [10., 11.],
        [ 8., 13.]])

In [29]:
test_approximately(matmul(A,B), (A@B))

In [30]:
matmul_einsum_time = timeit.timeit(partial(matmul,A,B), number=10)

## Improvement with einstein sum
How fast is matrix multiplication with einstein sum compared to array broadcasting?

In [31]:
broadcast_vs_einsum = matmul_broadcast_time/matmul_einsum_time
broadcast_vs_einsum

14.048343664400022

In [32]:
print(f"Einstein sum is about {broadcast_vs_einsum:.0f} times faster than array broadcasting")

Einstein sum is about 14 times faster than array broadcasting


## Linear Algebra Libraries

Using BLAS (Basic Linear Algebra Subprograms) in PyTorch

In [33]:
matmul_blas_time = timeit.timeit(partial(torch.matmul,A,B), number=10)

## Improvement with linear algebra libraries
How fast is matrix multiplication in PyTorch compared to doing it standard python?

In [34]:
loop_vs_blas = matmul_loop_time/matmul_blas_time
loop_vs_blas

37319.90540960608

In [35]:
print(f"Matrix multiplication in PyTorch is about {loop_vs_blas:.0f} times faster than using standard Python!")

Matrix multiplication in PyTorch is about 37320 times faster than using standard Python!


In [36]:
import jax, jax.numpy as jnp, time
key = jax.random.PRNGKey(0)
N = 4096
A = jax.random.uniform(key, (N, N), dtype=jnp.float32)
B = jax.random.uniform(key, (N, N), dtype=jnp.float32)

@jax.jit
def matmul(a,b): return a @ b

matmul(A,B).block_until_ready()  # warm-up/compile

t0 = time.perf_counter()
matmul(A,B).block_until_ready()
t1 = time.perf_counter()
print(f"TPU/JAX elapsed: {t1 - t0:.3f} s")


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


TPU/JAX elapsed: 0.001 s


___